<h1><center>Laboratorio 3: Clasificación y Clustering 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nelson Bruno Andrés Moreno Cabañas


### **Link de repositorio de GitHub:** `https://github.com/Bruno-Moreno/LabProgra`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Manejo de DataFrames con la libreria `pandas`.
- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar y aprovechar las ventajas que nos ofrece la libreria `pandas`.
- Utilizar `plotly` para obtener información gráfica del dataset.
- Aplicar las ventajas que nos ofrece crear un pipeline.
- Visualizar el funcionamiento de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

Requirement already up-to-date: plotly in c:\users\bruno\anaconda3\lib\site-packages (4.14.3)


# 1. Predicciones futboleras.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

Para este laboratorio se deberá trabajar con los csv ```FullData.csv```, donde deberán aplicar algoritmos de Clustering y clasificación información relevante de jugadores de futbol.

Para comenzar cargue el dataset señalado y visualice a traves de un head los atributos que posee el dataset.

In [2]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185 cm,80 kg,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170 cm,72 kg,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174 cm,68 kg,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182 cm,85 kg,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193 cm,92 kg,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183 cm,82 kg,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185 cm,80 kg,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173 cm,61 kg,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180 cm,80 kg,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


De la tabla se reconoce:

1. `Name`: Nombre del jugador
2. `Nationality`: Nacionalidad del jugador
3. `National_Position`: Posición en la selección, este valor solo existe cuando el jugador es seleccionado nacional.
4. `Club_Position`: Posición del jugador. Este parámetro tiene valor en gran parte de los jugadores, existen casos como `Res` o `Sub` que no los deberá considerar en la respectiva sección.
5. `Height`: Altura del jugador en centímetros. Si nota esta variable tiene un string, transforme a int.
6. `Weight`: Peso del jugador en kilogramos. Si nota esta variable tiene un string, transforme a int.
7. `Preffered_Foot`: Pie con que prefiere patear.
8. `Age`: Edad
9. `Work_Rate`: Frecuencia con que trabaja.

Mientras que el resto de las características representan estadísticas personales (de acuerdo a la habilidad) para cada uno de los jugadores.


## 1.1 Predicción de seleccionados nacionales. (2 puntos)

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


### 1.1.1 Generación de Labels para la Clasificación

Tras el trágico despido de la mítica mascota de la población de Maipú, *Renacín*, decide adentrarse en la predicción futbolera. Para esto, utilizando los datos cargados en las celdas anteriores, le consulta si es posible predecir **si un jugador de football será seleccionado nacional en base a sus estadísticas**

En este primer experimento de clasificación comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. Hecho esto, ¿Como se distribuyen los datos para cada clase?, comente lo que observa.


In [3]:
### Código aquí ###
def etiqueta(dato):
    if dato != 0:
        return 1
    else:
        return dato
    
df_players["National_Position"] = df_players["National_Position"].fillna(0).apply(etiqueta)

In [4]:
print(f"Cantidad de datos etiquetados como seleccionados (1): {(df_players.National_Position == 1).sum()}")
print(f"Cantidad de datos etiquetados como no-seleccionados (0): {(df_players.National_Position == 0).sum()}")

Cantidad de datos etiquetados como seleccionados (1): 1075
Cantidad de datos etiquetados como no-seleccionados (0): 16513


> Vemos en este caso 2 clases, aquellos jugadores seleccionados (1) con un total de 1075 y los jugadores no selecionados (2) con un total de 16513 jugadores. Las clases se encuentran muy desbalanceadas, con un ratio de 1:16 aproximadamente. 

### 1.1.2 Camino a la Clasificación

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="300">
</p>



Si bien los atributos de interés se encuentran en su mayoría limpios, existen valores mal ingresados al `DataFrame`. Para esto: 

1. Transforme a enteros todos los valores numéricos de la tabla que considere pertinentes. Por ejemplo: El peso aparece como strings y además estos continene la unidad. En este caso, debería transformar estos valores a enteros. Puede usar directamente funciones sobre el DataFrame (como `apply`) o el submódulo de las Series `str`.
2. Luego, aplique un ```ColumnTransformer``` para pre-procesar los datos del DataFrame de la siguiente forma:

Aplicar ```OneHotEncoder()``` sobre los atributos:

```python
['Preffered_Foot', 'Work_Rate']
```

Aplicar ```MinMaxScaler()``` sobre los atributos:

```python
['Age']
```

Aplicar ```StandardScaler()``` sobre los atributos:

```python
['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']
```

**Nota 2**: Puede copiar/pegar directamente las columnas en el `ColumnTransformer`.


Hecho lo anterior, defina al menos 3 pipelines para la clasificación donde anexe las transformaciones señaladas variando entre cada pipeline solo los clasificadores (puede escoger cualquiera de los clasificadores vistos en clases).

**To-Do:**

- [X] Modificar valores mal ingresados.
- [X] Aplicar ```ColumnTransformer``` sobre los datos a través de un pipeline.
- [X] Justificar porque se utilizan estos pre-procesamientos.
- [X] Crear 3 pipelines con diferentes clasificadores.
- [X] No entrenar los datos aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [5]:
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,1,LW,185 cm,80 kg,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,1,RW,170 cm,72 kg,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,1,LW,174 cm,68 kg,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,1,ST,182 cm,85 kg,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,1,GK,193 cm,92 kg,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11


In [18]:
df_players.dtypes

Name                 object
Nationality          object
National_Position     int64
Club_Position        object
Height                int64
Weight                int64
Preffered_Foot       object
Age                   int64
Work_Rate            object
Weak_foot             int64
Skill_Moves           int64
Ball_Control          int64
Dribbling             int64
Marking               int64
Sliding_Tackle        int64
Standing_Tackle       int64
Aggression            int64
Reactions             int64
Interceptions         int64
Vision                int64
Composure             int64
Crossing              int64
Short_Pass            int64
Long_Pass             int64
Acceleration          int64
Speed                 int64
Stamina               int64
Strength              int64
Balance               int64
Agility               int64
Jumping               int64
Heading               int64
Shot_Power            int64
Finishing             int64
Long_Shots            int64
Curve               

In [33]:
#Vemos la distribución de los datos en 
px.histogram(df_players["Age"],density=True)

TypeError: histogram() got an unexpected keyword argument 'density'

In [7]:
### Código aquí ###
#Transformamos a número las columnas de Altura y Peso 
df_players["Height"] = df_players["Height"].apply(lambda x: int(x.split()[0]))
df_players["Weight"] = df_players["Weight"].apply(lambda x: int(x.split()[0]))

In [74]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
#Ahora creamos el transformer y el pipeline
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(),  ['Preffered_Foot', 'Work_Rate']),
        ('StandardScaler', StandardScaler(),
         ['Height', 'Weak_foot', 'Skill_Moves', 
          'Ball_Control', 'Dribbling', 'Marking',
          'Sliding_Tackle', 'Standing_Tackle', 'Aggression',
          'Reactions','Interceptions', 'Vision', 'Composure',
          'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
          'Speed', 'Stamina', 'Strength', 'Balance', 'Agility',
          'Jumping', 'Heading', 'Shot_Power', 'Finishing', 
          'Long_Shots', 'Curve', 'Freekick_Accuracy',
          'Penalties', 'Volleys']), 
        ('MinMaxScaler', MinMaxScaler(), ['Age'])])

#Creamos ahora los pipelines 
Pipe1 = Pipeline([('Preprocessing', preprocessing_transformer),('SVC', SVC())])
Pipe2 = Pipeline([('Preprocessing', preprocessing_transformer),('NB', GaussianNB())])
Pipe3 = Pipeline([('Preprocessing', preprocessing_transformer),('LogReg', LogisticRegression(max_iter = 1000))])

> Se realizó un cambio en los valores para Height y Weight para que fueran reconocidos efectivamente como valores, por otro lado, para las variables categóricas se utilizó un one hot encoding para pasarla a valores numéricos (no son ordinales como para haber utilizado un labelencoder), para las variables numéricas bien portadas (gaussianas) se utilizó un escalamiento standard para llevar los datos a media 0 y desviación 1 , finalmente para la variable "Age" se llevó al rango 0,1 pues como se puede ver en la gráfica, no tiene un comportamiento gaussiano y nos gustaría mantener la proporción de distancias entre los datos.

### 1.1.3 Métricas de evaluación + Opcional

<p align="center">
  <img src="https://media4.giphy.com/media/3orif8r5vpbAdQU3iU/200w.gif?cid=82a1493btc1h43n1h4c59p13kjv6adugso4lo4292zbgt66b&rid=200w.gif&ct=g" width="300">
</p>

Interesado por saber que tan bien funciona su clasificador, Renacin le solicita que construya **a mano** las siguientes métricas de evaluación para el caso de **clasificación binaria**:

- Accuracy (Exactitud):

$$Accuracy = \dfrac{número \, de \, predicciones \, correctas}{número \, de \,predicciones \, totales}$$


- Precision: 

$$Precision = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \, positivos}$$

- Recall:

$$Recall = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \,negativos}$$

<p align="center">
  <img src="https://www.researchgate.net/publication/336402347/figure/fig3/AS:812472659349505@1570719985505/Calculation-of-Precision-Recall-and-Accuracy-in-the-confusion-matrix.ppm" width="350">
</p>

**To-Do:**

- [X] Crear funciones para las métricas para clasificación binaria: Accuracy, Precision y Recall. 
- [X] Compruebe el correcto funcionamiento de sus métricas a través de los test.

**Hint:** Utilice confusion_matrix para obtener TP, FP, FN y TN.



In [88]:
from sklearn.metrics import confusion_matrix 
def accuracy(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    return (tp+tn)/(tn+fp+fn+tp)

def precision(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    if tp == 0:
        return 0
    else:
        return (tp)/(tp+fp)

def recall(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    return (tp)/(tp+fn)


In [89]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

y_true = [0, 1, 0, 0, 1, 0]
y_pred = [0, 0, 1, 0, 0, 1]

# Tests de metricas
assert accuracy_score(y_true, y_pred) == accuracy(y_true, y_pred)
assert recall_score(y_true, y_pred) == recall(y_true, y_pred)
assert precision_score(y_true, y_pred) == precision(y_true, y_pred)

### 1.1.4 Entrenemos los pipelines

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="300">
</p>


Entrene en base al pipeline generado en los pasos anteriores, procure separar los datos de entrenamiento en un conjunto de entrenamiento y de prueba (la proporción queda a su juicio). Una vez entrenado, muestre los resultados para cada uno de los clasificadores y comente lo que observa de los resultados obtenidos usando las métricas creadas anteriormente, ¿podrían mejorarse los resultados?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar el pipeline.
- [X] Utilizar las métricas codificadas anteriormente y analizar los resultados.


In [110]:
### Código aquí ###
#Separamos la data 
X_train, X_test, y_train, y_test = train_test_split(df_players.drop(columns = ['National_Position']),
                                                    df_players["National_Position"], shuffle=True, 
                                                   test_size=0.3, random_state=666 , stratify = df_players["National_Position"])
#Revisamos el score de cada uno de los modelos para el conjunto de test 
#Pipe1.fit(X_train, y_train)
#Pipe2.fit(X_train, y_train)
#Pipe3.fit(X_train, y_train)
#print(f'SVC score en test: {Pipe1.score(X_test,y_test)}') #Accuracy
#print(f'GB score en test: {Pipe2.score(X_test,y_test)}') #Accuracy
#print(f'LogReg score en test: {Pipe3.score(X_test,y_test)}') #Accuracy

#Hacemos fit a los modelos 
Pipe1.fit(X_train, y_train)
Pipe2.fit(X_train, y_train)
Pipe3.fit(X_train, y_train)
#Hacemos las predicciones de cada uno de los modelos

y_pred1 = Pipe1.predict(X_test)
y_pred2 = Pipe2.predict(X_test)
y_pred3 = Pipe3.predict(X_test)

#Definimos el y_true
y_true = y_test

#Calculamos oas accuracy, recall y precision
acc1 = accuracy(y_true,y_pred1)
acc2 = accuracy(y_true,y_pred2)
acc3 = accuracy(y_true,y_pred3)

recall1 = recall(y_true,y_pred1)
recall2 = recall(y_true,y_pred2)
recall3 = recall(y_true,y_pred3)

precision1 = precision(y_true,y_pred1)
precision2 = precision(y_true,y_pred2)
precision3 = precision(y_true,y_pred3)

d = {'SVC': [acc1, recall1, precision1], 'NB': [acc2,recall2,precision2] , 'LogReg': [acc3,recall3,precision3]}
resultados = pd.DataFrame(data=d)
resultados = resultados.rename(index = {0: "Accuracy" , 1: "Recall" , 2: "Precision" } )
resultados



,SVC,NB,LogReg
Accuracy,0.938791,0.796286,0.940307
Recall,0.000000,0.510836,0.055728
Precision,0.000000,0.152495,0.642857


**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```
> Vemos que los modelos tienen una alta accuracy en general pero dado que las clases se encuentran tan desbalanceadas, los modelos predicen en su mayoría 0's. En este sentido, si nos interesa predecir cuales son los jugadores seleccionados nacionales, entonces tenemos que fijarnos en las métricas Recall y Precision pues están consideran la clase positiva 1. El mejor resultado en este caso lo tiene el Gaussian Naive Bayes con el mayor valor para Recall, mientras que para Precision el lugar se lo lleva LogReg. Para mejorar los resultados, es necesario tener más datos de la clase 1 en el conjunto de train lo cual se puede hacer con subsampling u oversampling.

## 1.2 Predicción de posiciones de jugadores. (2 puntos)

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

### 1.2.1 Creación de nuevas labels.



No conforme con el trabajo de la sección 1.1, Renacin les *ordena* que construyan un clasificador nuevo, pero para predecir la posición de los jugadores del dataset. Para lograr esto, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, con esto les recalca que no deben utilizar los valores ```Sub``` y ```Res``` de este atributo, pidiendo que agrupen los valores en los siguientes grupos:

```python
Ataque=['ST', 'CAM','RS','LF','LAM','LS','RF','RAM','CF']
MedioCampistas=['RCM','LW', 'RW', 'LCM','LM', 'RM', 'CM']
Defensas=['LCB','RB','RCB', 'LDM','LB','CDM','RDM','CB','RWB','LWB']
Arquero =['GK']
```

Con lo señalado, Renacin les señala que no deberian utilizar el label utilizado en la sección 1.1, ya que el problema de clasificación ahora es de posiciones.

**To-Do**:

- [ ] Aplique las etiquetas de ```Ataque```, ```MedioCampista```, ```Defensa``` y ```Arquero``` en los valores señalados en esta sección.
- [ ] Muestre como se distribuyen los datos etiquetados.

In [ ]:
print(f"Cantidad de datos etiquetados como delantero: {(df_players.Club_Position == 'Delantero').sum()}")
print(f"Cantidad de datos etiquetados como defensa: {(df_players.Club_Position == 'Defensa').sum()}")
print(f"Cantidad de datos etiquetados como medio campistas: {(df_players.Club_Position == 'Medio').sum()}")
print(f"Cantidad de datos etiquetados como arqueros: {(df_players.Club_Position == 'Arquero').sum()}")

### 1.2.2 Entrenar un pipeline

Aplicando el mismo preprocesamiento solicitado en 1.1.2, genere un pipeline con un clasificador a gusto y evalúe al modelo de clasificación utilizando la **librería `scikit-learn`** (esto debido a que este es un problema multi clase y las funciones solicitadas anteriormente era para una clasificación binaria).

De los resultados obtenidos, ¿El modelo pudo clasificar mejor este problema o el de la sección 1.1?, Comente sus resultados.

**To-Do**:

- [ ] Genere un pipeline similar al visto en la sección 1.1.2.
- [ ] Aplique las métricas ofrecidas por la librería scikit-learn.
- [ ] Responda la pregunta del enunciado.

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados.
```

## 1.3 Clustering Sobre el Dataset (2 puntos)

<p align="center">
  <img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1832816%2F349289c622faaff3afc517a46fcf4c71%2Fk-means.jpg?generation=1577258176400530&alt=media" width="200">
</p>

### 1.3.1 Elección de numero de clusters

<p align="center">
  <img src="https://media1.tenor.com/images/a85e9df2f5784d6513dce853ba9d5755/tenor.gif?itemid=18229271" width="200">
</p>





Entrene un k-means con 3, 4 y 5 clusters. ¿Las agrupaciones son lógicas usando un análisis visual?, Grafique los clusters con el código dispuesto más abajo y comente cual de las opciones presenta una mejor agrupación.

En base al análisis realizado, ubique la posición de los cluster en la cancha. Solo señale a que posición equivaldría tomando de referencia:

- Ataque
- Medio campistas
- Defensas
- Arquero 

**To-Do**:

- [ ] Escoger un numero de cluster y justificar elección.
- [ ] Comentar si las agrupaciones son lógicas a través de un análisis visual.
- [ ] Graficar los resultados obtenidos y visualizar las agrupaciones obtenidas.


In [ ]:
from umap import UMAP

def graficar_clusters(clusters, preprocessed_data = preprocessed_data,posicion = df_players.Club_Position.values):
  # Entreguele a esta función solo los clusters, el resto de parametros debería funcionar por default.
  umap = UMAP()
  projections = umap.fit_transform(preprocessed_data)
  fig = px.scatter(x=projections[:,0], y=projections[:,1], color=clusters, hover_data={'posicion': posicion}) 
  fig.show()

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados.
```

### 1.3.2 Búsqueda Número Óptimo de Clusters: Método del Codo **(Opcional 0.5 puntos)**

<p align="center">
  <img src="https://c.tenor.com/fk4p5HzTi5gAAAAC/elbow-high-five.gif" width="300">
</p>





Renancin muy indeciso de sus resultados le comenta que desea realizar un último trabajo. En esta última tarea le solicita la agrupación de los datos obtenidos en la sección 1.2 utilizando K-means. Para esto usted deberá visualizar a través del método del codo el número óptimo de agrupaciones para los datos. 

**Hint:** Revisar las clases y encontrar el código.

**To-Do**:

- [ ] Pre-procesar los datos con el mismo preprocesamiento aplicado en 1.2
- [ ] Utilizar el método del codo para obtener el número de clusters.

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>